# 環境構築

In [ ]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7

In [1]:
from google.colab import drive
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import pickle
# import MeCab
import pandas as pd
#wakati = MeCab.Tagger("-Owakati")

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


# Dock2Vecに対応したデータへの前処理

In [131]:
sentences = []
advise = []
dictionary = {}
send_ms = '朝食は抜きでした。昼食は、ラーメンとご飯を食べました。午前、午後共に、車での移動なので運動は特になし。夕食はスープとハヤシライスでした。運動は家に帰ってからの犬の散歩と、階段の上り下りです。明け方まで起きていたので、夜食に焼きそばを食べました。'
sentence_num = 0
with open('drive/My Drive/Health/AfterPreprocessingData.tsv', 'r', encoding='utf-8') as f:
  lines = f.readlines()
  for line in lines:
    # 配列から##と\n文字の削除
    line = line.replace('##', '')
    line = line.replace('\n', '')
    
    # 同じ文書だった場合、辞書にレスポンスを追加
    if send_ms == line[:line.find('|')]:
      dictionary.setdefault(sentence_num, []).append(line[line.find('|')+1:])
    else:
      sentences.append(send_ms)
      sentence_num += 1
    send_ms = line[:line.find('|')]
      

In [132]:
print(sentences[0])
print(dictionary[1])

朝食は抜きでした。昼食は、ラーメンとご飯を食べました。午前、午後共に、車での移動なので運動は特になし。夕食はスープとハヤシライスでした。運動は家に帰ってからの犬の散歩と、階段の上り下りです。明け方まで起きていたので、夜食に焼きそばを食べました。
['何かと忙しい中、食事を摂るだけでもたいへんかと思います。', 'お相撲さんは、太るために朝ご飯をたべずに、昼ごはんをたくさんたべるそうです。', '果物でビタミンＣも補給してあげて下さいね。', '筋肉量を増やすのにタンパク質は欠かせません。', '今の季節、鍋料理はよいですね。', '朝食を抜くと体は脂肪を溜め込もうと働きます。', 'とても健康に気を使われているのがよくわかります。', '今日はボリュームのある昼食でしたね。', '人間の体の約６０％は水分です。', '便秘解消には食物繊維が不可欠です。', '痩せたいのであれば、一日三食はやはり基本です。', '炭水化物と炭水化物という組み合わせではなく、炭水化物（ご飯・パン・めん）とおかずという組み合わせにすると栄養バランスがとれますよ。', '朝の果物はおすすめですよ。', '生活習慣病（がん・脳血管疾患・心疾患・糖尿病など）はその名の通り生活習慣が原因でかかる病気です。', '一日に必要な野菜の量は、約３５０ｇです。', '量が少なくても、おなかを満たすコツはよく噛んで時間をかけて食べる事です。', '冷たいものを飲むと、体は勿論冷えます。', '朝から準備されるのは大変なので、前日の夕食や常備菜をまとめて作って活用してはいかがですか？', 'ダイエットの基本は、消費カロリーが摂取カロリーを上回ることです。', '冬でも水分補給は必要です。', '果物をデザートに選ばれてはどうでしょうか。', '外食はどうしても野菜不足になりがちです。', '脳は、ブドウ糖しかエネルギーとして使うことができません。', 'コーヒーをよく飲まれるんですね。', '甘いものは夕食後は避けた方がいいかもしれませんね。', '今日は、朝食もきちんと摂られてすばらしいです。', '朝食がなかなかすすまないようですね。', '昼食、夕食が同じメニューだとやはり栄養が偏ってしまいますね。', '規則正しい食生活は健康的ですね。', '冬野菜（大根・白菜・白ねぎ等）には体を温める効果がありま

In [ ]:
outfname = "drive/My Drive/Health/Dock2VecDataSet.tsv"

## ファイルに保存

In [ ]:
# 文章の重複削除
sentences = list(set(sentences))

In [ ]:
outfp = open(outfname, "w")
for text in sentences:
  text = text + "\n"
  outfp.write(text)
outfp.close()

# ファイルの読み込み & sentenceの作成

In [ ]:
sentences = []
test = []
with open('drive/My Drive/Health/Dock2VecDataSet.tsv', 'r', encoding='utf-8') as f:
  lines = f.readlines()
  for line in lines:
    line = line.replace('\n', '')
    # 分かち書きを行う
    test = wakati.parse(line).split()
    sentences.append(test)

In [ ]:
print(sentences[0])
print(sentences[1])

['昨日', 'は', '朝', '、', '寝', 'て', 'た', 'ため', 'に', '朝', 'ごはん', 'は', '食べ', 'て', 'ませ', 'ん', '。', '日', '中', 'は', '掃除', 'と', '片付け', 'を', 'し', 'て', '、', '夕方', 'ハンバーグ', 'を', '食べ', 'まし', 'た', '。', '体調', 'が', '悪かっ', 'た', 'ので', '、', 'なるべく', 'ゆっくり', '過ごし', 'まし', 'た', '。']
['朝', 'ごはん', 'は', '食べ', 'ず', 'に', '、', '午前', '中', 'は', '掃除', 'を', '行い', 'まし', 'た', '。', 'お昼', 'ごはん', 'に', '、', 'チャプチェ', 'と', 'ご飯', '、', '小', '芋', 'の', '煮物', 'と', '玉ねぎ', 'の', '味噌汁', 'を', '食べ', 'まし', 'た', '。', '午後', 'から', 'は', '、', 'ゆっくり', '過ごし', 'まし', 'た', '。', '晩', 'ご飯', 'に', '、', 'きつね', 'うどん', 'と', '椎茸', 'の', 'ソテー', '、', '茎', 'わかめ', 'の', 'サラダ', 'を', '食べ', 'まし', 'た', '。', '夜', 'は', '、', '犬', 'の', '散歩', 'に', '１', '時間', 'ほど', '行き', 'まし', 'た', '。', 'その', '際', '、', '階段', 'の', '上り', '下り', 'を', '５', '階', '分', 'し', 'て', 'い', 'ます', '。']


# Dock2Vecを使用してモデルの作成

In [ ]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(sentences)]
model = Doc2Vec(documents, vector_size=100, window=5, min_count=1, epochs=10, workers=4)

In [ ]:
print(documents)
# f = open('drive/My Drive/Health/sample.binaryfile', 'wb')
# pickle.dump(model,f)
# f.close

[TaggedDocument(words=['昨日', 'は', '朝', '、', '寝', 'て', 'た', 'ため', 'に', '朝', 'ごはん', 'は', '食べ', 'て', 'ませ', 'ん', '。', '日', '中', 'は', '掃除', 'と', '片付け', 'を', 'し', 'て', '、', '夕方', 'ハンバーグ', 'を', '食べ', 'まし', 'た', '。', '体調', 'が', '悪かっ', 'た', 'ので', '、', 'なるべく', 'ゆっくり', '過ごし', 'まし', 'た', '。'], tags=[0]), TaggedDocument(words=['朝', 'ごはん', 'は', '食べ', 'ず', 'に', '、', '午前', '中', 'は', '掃除', 'を', '行い', 'まし', 'た', '。', 'お昼', 'ごはん', 'に', '、', 'チャプチェ', 'と', 'ご飯', '、', '小', '芋', 'の', '煮物', 'と', '玉ねぎ', 'の', '味噌汁', 'を', '食べ', 'まし', 'た', '。', '午後', 'から', 'は', '、', 'ゆっくり', '過ごし', 'まし', 'た', '。', '晩', 'ご飯', 'に', '、', 'きつね', 'うどん', 'と', '椎茸', 'の', 'ソテー', '、', '茎', 'わかめ', 'の', 'サラダ', 'を', '食べ', 'まし', 'た', '。', '夜', 'は', '、', '犬', 'の', '散歩', 'に', '１', '時間', 'ほど', '行き', 'まし', 'た', '。', 'その', '際', '、', '階段', 'の', '上り', '下り', 'を', '５', '階', '分', 'し', 'て', 'い', 'ます', '。'], tags=[1]), TaggedDocument(words=['朝食', 'は', '食パン', '２', '枚', '食べ', 'まし', 'た', '。', '通勤', 'の', 'ため', '自転車', 'に', '１', '０', '分', 'ほど', '乗り', 'まし', 'た',

In [ ]:
f = open('drive/My Drive/Health/sample.binaryfile','rb')
model = pickle.load(f)

In [ ]:
# 朝食と昼食の類似度
print(model.docvecs.most_similar(0))
print(model.docvecs.similarity(0,26))

[(71, 0.9862009882926941), (19, 0.9856235384941101), (18, 0.9855508804321289), (136, 0.9853261113166809), (244, 0.9852647185325623), (442, 0.9852074384689331), (130, 0.9849938154220581), (273, 0.9849393367767334), (90, 0.9848341941833496), (174, 0.9848005771636963)]
0.9819067


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [ ]:
print(sentences[0])
print(sentences[71])

['昨日', 'は', '朝', '、', '寝', 'て', 'た', 'ため', 'に', '朝', 'ごはん', 'は', '食べ', 'て', 'ませ', 'ん', '。', '日', '中', 'は', '掃除', 'と', '片付け', 'を', 'し', 'て', '、', '夕方', 'ハンバーグ', 'を', '食べ', 'まし', 'た', '。', '体調', 'が', '悪かっ', 'た', 'ので', '、', 'なるべく', 'ゆっくり', '過ごし', 'まし', 'た', '。']
['朝', 'ごはん', 'は', '、', '食べ', 'て', 'い', 'ませ', 'ん', '。', '昼', 'ごはん', 'は', '、', 'マック', 'で', 'セット', '。', '夜', 'ごはん', 'は', '、', 'カツ', '丼', 'に', 'サラダ', 'に', 'コロッケ', 'に', 'ビール', '。', '運動', 'は', '、', 'とくに', 'し', 'て', 'い', 'ませ', 'ん', '。']


# 自分が予想される文章を作成して、返答が返されるかテスト

##テストデータの前処理

In [ ]:
sentences = []
with open('drive/My Drive/Health/Test.tsv', 'r', encoding='utf-8') as f:
  lines = f.readlines()
  for line in lines:
    # 配列から##と\n文字の削除
    '''
    # センドメッセージとリプライメッセージに分割する場合
     「|」文字で分割する
     text_list = line.split('|')
     sentences.append(text_list)
    '''
    # センドメッセージのみを使用
    line = line[:line.find('|')]
    sentences.append(line)
# 文章の重複削除
sentences = list(set(sentences))

In [ ]:
print(sentences)

['今日は起床が遅く、朝食をとる時間が無く食べませんでした。昼はコンビニで買ったサンドイッチを食べました。夜は野菜を多めに摂りました。', '疲れが溜まっており早めに寝た。仕事が朝早くから有った為、いつもより早めに起きた。仕事がキツイ為、しっかり3食食べた。', '今日は１日何も食べずに過ごした。運動もせず、1日中家でゲームをしていた。起床,睡眠 時間はいつもどうり、早寝早起きをした。', '朝7時 朝食。昼12時 昼食。夜20時 夜食。3食ともに野菜を食べた。睡眠時間 7時間。']


In [ ]:
outfname = "drive/My Drive/Health/TestDataSet.tsv"

In [ ]:
outfp = open(outfname, "w")
for text in sentences:
  text = text + "\n"
  outfp.write(text)
outfp.close()

## テストデータでセンテンス作成

In [ ]:
sentences = []
test = []
with open('drive/My Drive/Health/TestDataSet.tsv', 'r', encoding='utf-8') as f:
  lines = f.readlines()
  for line in lines:
    line = line.replace('\n', '')
    # 分かち書きを行う
    test = wakati.parse(line).split()
    sentences.append(test)

In [ ]:
print(sentences[0])
print(sentences[1])
print(len(sentences))

['今日', 'は', '起床', 'が', '遅く', '、', '朝食', 'を', 'とる', '時間', 'が', '無く', '食べ', 'ませ', 'ん', 'でし', 'た', '。', '昼', 'は', 'コンビニ', 'で', '買っ', 'た', 'サンドイッチ', 'を', '食べ', 'まし', 'た', '。', '夜', 'は', '野菜', 'を', '多め', 'に', '摂り', 'まし', 'た', '。']
['朝', '7', '時', '朝食', '。', '昼', '12', '時', '昼食', '。', '夜', '20', '時', '夜食', '。', '3', '食', 'とも', 'に', '野菜', 'を', '食べ', 'た', '。', '睡眠', '時間', '7', '時間', '。']
4


# most_similarを使用

In [ ]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(sentences)]
print(documents)
model = Doc2Vec(documents, vector_size=100, window=5, min_count=1, epochs=100, workers=4)

[TaggedDocument(words=['今日', 'は', '起床', 'が', '遅く', '、', '朝食', 'を', 'とる', '時間', 'が', '無く', '食べ', 'ませ', 'ん', 'でし', 'た', '。', '昼', 'は', 'コンビニ', 'で', '買っ', 'た', 'サンドイッチ', 'を', '食べ', 'まし', 'た', '。', '夜', 'は', '野菜', 'を', '多め', 'に', '摂り', 'まし', 'た', '。'], tags=[0]), TaggedDocument(words=['朝', '7', '時', '朝食', '。', '昼', '12', '時', '昼食', '。', '夜', '20', '時', '夜食', '。', '3', '食', 'とも', 'に', '野菜', 'を', '食べ', 'た', '。', '睡眠', '時間', '7', '時間', '。'], tags=[1]), TaggedDocument(words=['疲れ', 'が', '溜まっ', 'て', 'おり', '早め', 'に', '寝', 'た', '。', '仕事', 'が', '朝', '早くから', '有っ', 'た', '為', '、', 'いつも', 'より', '早め', 'に', '起き', 'た', '。', '仕事', 'が', 'キツイ', '為', '、', 'しっかり', '3', '食', '食べ', 'た', '。'], tags=[2]), TaggedDocument(words=['今日', 'は', '１', '日', '何', 'も', '食べ', 'ず', 'に', '過ごし', 'た', '。', '運動', 'も', 'せ', 'ず', '、', '1', '日', '中家', 'で', 'ゲーム', 'を', 'し', 'て', 'い', 'た', '。', '起床', ',', '睡眠', '時間', 'は', 'いつも', 'どう', 'り', '、', '早寝', '早起き', 'を', 'し', 'た', '。'], tags=[3])]


In [ ]:
def process_query(query):
  text_wakati = wakati.parse(query).split()
  return text_wakati

query = "今日は起床が遅く、朝食をとる時間が無く食べませんでした。昼はコンビニで買ったサンドイッチを食べました。夜は野菜を多めに摂りました。"
l = process_query(query)
sim = model.most_similar(positive=l,topn=10)
print(sim)

# 実際にモデルを使用して類似度検索を行う

In [ ]:
# 朝食と昼食の類似度
print(model.docvecs.most_similar(0))
print(model.docvecs.similarity(0,0))

[(3, 0.9993677139282227), (2, 0.9992536306381226), (1, 0.9989633560180664)]
1.0


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [ ]:
def SameSentence(text):
    text_wakati = wakati.parse(text).split()
    print(text_wakati)
    # documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(text_wakati)]
    # print(documents)
    # similarity = model.similarity(documents,0)
    # print(model.docvecs.most_similar(["myfizzbuzz.py"]))
    # test = model.most_similar(documents)
    test = model.most_similar(text_wakati)
    return test

print(SameSentence("今日は起床が遅く、朝食をとる時間が無く食べませんでした。昼はコンビニで買ったサンドイッチを食べました。夜は野菜を多めに摂りました。"))

['今日', 'は', '起床', 'が', '遅く', '、', '朝食', 'を', 'とる', '時間', 'が', '無く', '食べ', 'ませ', 'ん', 'でし', 'た', '。', '昼', 'は', 'コンビニ', 'で', '買っ', 'た', 'サンドイッチ', 'を', '食べ', 'まし', 'た', '。', '夜', 'は', '野菜', 'を', '多め', 'に', '摂り', 'まし', 'た', '。']
[('ず', 0.998121440410614), ('いつも', 0.9977579116821289), ('時', 0.9977397918701172), ('過ごし', 0.9977344274520874), ('為', 0.9975861310958862), ('仕事', 0.9975199103355408), ('早め', 0.9973146915435791), ('ゲーム', 0.9972864389419556), ('日', 0.9971828460693359), ('より', 0.9971119165420532)]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [ ]:
# 学習データの読み込み
df = pd.read_csv('drive/My Drive/Health/TestDataSet01.tsv')
df = df.dropna()

In [ ]:
def ans(text, df):
  text_wakati = wakati.parse(text).split()
  wmd = lambda x: model.wv.wmdistance(text_wakati, x)
  print(wmd)
  result = df['input_wakati'].map(wmd).idxmin()
  return df['output'].iloc[result]

In [ ]:
test = ans("私は錘大学の学生です。", df)

In [ ]:
x = model.infer_vector(wakati.parse("今日は起床が遅く、朝食をとる時間が無く食べませんでした。昼はコンビニで買ったサンドイッチを食べました。夜は野菜を多めに摂りました。").split())
print(model.docvecs.most_similar([x]))

[(0, 0.9997673034667969), (3, 0.9997329711914062), (2, 0.9995917081832886), (1, 0.9993623495101929)]


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [ ]:
x = model.infer_vector(wakati.parse("どうも井熊です。").split())
print(model.docvecs.most_similar([x]))

[(1, 0.6589229702949524), (3, 0.6578977108001709), (2, 0.6569473743438721), (0, 0.6563471555709839)]


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


# 角田先生案

## 原型プログラム

In [ ]:
def noda(text):
  text_wakati = wakati.parse(text).split()
  return text_wakati


test = noda("今日は起床が遅く、朝食をとる時間が無く食べませんでした。昼はコンビニで買ったサンドイッチを食べました。夜は野菜を多めに摂りました。")

for i in range():
  sim_value = model.docvecs.similarity_unseen_docs(model, test, sentences[0], alpha=1, min_alpha=0.0001, steps=5)
  print(sim_value)

0.999997


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


## 角田先生案 コードのリファクタリング

In [ ]:
def ResponseText(text):
  max_value = 0
  max_num = 0
  text_wakati = wakati.parse(text).split()
  for i in range(len(sentences)):
    sim_value = model.docvecs.similarity_unseen_docs(model, text_wakati, sentences[i], alpha=1, min_alpha=0.0001, steps=5)
    if max_value < sim_value:
      max_value = sim_value
      max_num = i
    #max_value = max(max_value, sim_value)
  return sentences[max_num]
  #類似度Max
  #return sentences[i]




"""
while True:
  text = input(">>>")
  if text == "終了":
    break
"""

anser = ResponseText("今日は起床が遅く、朝食をとる時間が無く食べませんでした。昼はコンビニで買ったサンドイッチを食べました。夜は野菜を多めに摂りました。")
print(ansert)